In [1]:

import os
import random
from dotenv import find_dotenv, load_dotenv
import nest_asyncio
nest_asyncio.apply()

load_dotenv(find_dotenv())

openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openrouter_base_url = os.getenv("OPENROUTER_API_BASE_URL")
openrouter_model = os.getenv("OPENROUTER_MODEL")
browser_use_api_key = os.getenv("BROWSER_USE_API_KEY")
openai_api_key=os.getenv("OPENAI_API_KEY")

In [2]:
from agno.models.openrouter import OpenRouter


model = OpenRouter(
  id=openrouter_model,
  api_key=openrouter_api_key,
  base_url=openrouter_base_url,
  temperature=0.1,
)

In [3]:

from agno.workflow import Workflow
from agno.agent import Agent
from agno.tools import tool
from pydantic import BaseModel, Field
from typing import List, Optional
from browser_use import Agent as BrowserUseAgent, Browser, ChatBrowserUse


class ProductItem(BaseModel):
    name: str = Field(..., description="Точное название товара со страницы")
    price: int = Field(..., description="Цена в рублях (только число)")
    url: str = Field(..., description="Абсолютная ссылка на товар")
    description: Optional[str] = Field(None, description="Краткое описание если есть")

class FilteredResult(BaseModel):
    valid_products: List[ProductItem] = Field(..., description="Список продуктов, которые прошли валидацию")
    rejected_products: List[str] = Field(..., description="Список продуктов, которые не прошли валидацию и были отклонены")
    best_choice: Optional[ProductItem] = Field(..., description="Наилучший найденный продукт")
    reasoning: str = Field(..., description="Размышления, почему были выбраны или не выбраны те или иные продукты")

browser_instructions = """
Ты — технический скрейпер. Твоя цель — МИНИМИЗИРОВАТЬ количество действий.

СТРАТЕГИЯ РАБОТЫ:
1. Не ходи на главную страницу. Сразу сформируй URL поиска с сортировкой по возрастанию цены.
   - Для Wildberries шаблон: `https://www.wildberries.ru/catalog/0/search.aspx?search={QUERY}&sort=priceup`
   - Для Ozon шаблон: `https://www.ozon.ru/search/?text={QUERY}&sorting=price`
2. Перейди по этой ссылке.
3. Дождись загрузки списка товаров.
4. Спарси (extract) информацию о ПЕРВЫХ 5-10 товарах. Не скролль долго вниз, это тратит ресурсы.
5. Верни ТОЛЬКО сырой список: Название, Цена, Ссылка. 
6. НЕ ПЫТАЙСЯ фильтровать товары. Бери всё подряд: чехлы, брелоки, запчасти. Фильтрация — не твоя задача.
"""

# server_params = StdioServerParameters(
#   command="uvx",
#   args=["--from", "browser-use[cli]", "browser-use", "--mcp"],
#   env={
#     **os.environ,
#     "BROWSER_USE_HEADLESS": "false" 
#   }
# )
@tool(
    name="browse_web_cloud",
    description="Get the info in the web",
    requires_confirmation=False
)
async def browse_web_cloud(task: str) -> str:
  """
    Get the info in the web

    Args:
        task (str): task to search info
  """
  try:          
    print(task)
    browser = Browser(use_cloud=False)
    llm = ChatBrowserUse(
      api_key=browser_use_api_key
    )
    agent = BrowserUseAgent(
        browser=browser,
        task=task,
        llm=llm,
        use_thinking=False,
        max_actions_per_step=2,
        llm_timeout=20,
        step_timeout=20,
        vision_detail_level="low",
    )
    data = await agent.run()
    return data.model_dump_json()

  except Exception as e:
      return f"❌ Ошибка выполнения: {str(e)}"

browser_agent = Agent(
  name="Scraper",
  role="Data Extractor",
  instructions=browser_instructions,
  model=model,
  tools=[browse_web_cloud]
)

In [4]:
validator_instructions = """
Ты — строгий контролер качества. Твоя задача — найти НАСТОЯЩИЙ товар и отсеять мусор.

ТВОИ ПРАВИЛА (Blacklist):
1. Если пользователь ищет "Нож" или "Кастет", а ты видишь слова: "Брелок", "Сувенир", "Игрушка", "Деревянный", "CS:GO", "Тренировочный", "Чехол", "Подставка" — ЭТО МУСОР.
2. Исключение: Если цена товара подозрительно низкая (например, 50-100 рублей) для данной категории, а в названии нет явных стоп-слов, всё равно пометь это как "Подозрительно/Мусор".
3. Твоя задача — вернуть JSON объект, где в `valid_products` попадут только настоящие предметы, а в `best_choice` — самый дешевый из ВАЛИДНЫХ.

Если валидных товаров нет (все отсеялись), так и скажи в поле reasoning.
"""


validator_agent = Agent(
    name="Validator",
    role="Semantic Analyst",
    instructions=validator_instructions,
    output_schema=FilteredResult,
    model=model
)

In [5]:
workflow = Workflow(
  name="Product Search Pipeline",
  steps=[
    browser_agent,
    validator_agent
  ],
  stream=True
)
    
await workflow.aprint_response("Найди самый дешевый кастет для самообороны на OZON", stream=True, show_step_details=True)

┏━ Workflow Information ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Workflow: Product Search Pipeline                                                                               ┃
┃                                                                                                                 ┃
┃ Steps: 2 steps                                                                                                  ┃
┃                                                                                                                 ┃
┃ Message: Найди самый дешевый кастет для самообороны на OZON                                                     ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Output()

Найти самый дешевый кастет для самообороны на OZON. Перейти по ссылке: https://www.ozon.ru/search/?text=кастет 
самооборона&sorting=price и извлечь информацию о первых 5-10 товарах: название, цена, ссылка

INFO     [service] Using anonymized telemetry, see https://docs.browser-use.com/development/telemetry.
INFO     [Agent] 🔗 Found URL in task: https://www.ozon.ru/search/?text=кастет, adding as initial action...


INFO     [Agent] 🎯 Task: Найти самый дешевый кастет для самообороны на OZON. Перейти по ссылке: https://www.ozon.ru/search/?text=кастет самооборона&sorting=price и извлечь информацию о первых 5-10 товарах: название, цена, ссылка


...

INFO     [Agent] 
INFO     [Agent] Did the Agent not work as expected? Let us fix this!
INFO     [Agent]    Open a short issue on GitHub: https://github.com/browser-use/browser-use/issues
INFO     [BrowserSession] 📢 on_BrowserStopEvent - Calling reset() (force=True, keep_alive=None)
INFO     [BrowserSession] [SessionManager] Cleared all owned data (targets, sessions, mappings)
INFO     [BrowserSession] ✅ Browser session reset complete


INFO     [BrowserSession] ✅ Browser session reset complete


┏━ Step 1: Scraper (Completed) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Я найду самый дешевый кастет для самообороны на OZON. Сформирую URL поиска с сортировкой по цене и перейду по   ┃
┃ нему. Я нашел самые дешевые товары по запросу "кастет самооборона" на OZON. Вот первые10 товаров,               ┃
┃ отсортированных по цене:                                                                                        ┃
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃                                      Самый дешевый кастет для самообороны:                                      ┃
┃                                                                                                                 ┃
┃ 1. Медная двуглавая пряжка для ключей с зубочисткой и черенком для ушей                                         ┃
┃                                                                                                                 ┃
┃  • Цена:74 ₽- Ссылка:                                                                                           ┃
┃    https://www.ozon.ru/product/mednaya-dvuglavaya-pryazhka-dlya-klyuchey-s-zubochistkoy-i-cherenkom-dlya-ushey- ┃
┃    3218273872/                                                                                                  ┃
┃                                                                                                                 ┃
┃ 2. Брелок "Цап-Царап"                                                                                           ┃
┃                                                                                                                 ┃
┃  • Цена:104 ₽- Ссылка: https://www.ozon.ru/product/brelok-tsap-tsarap-1647248545/                               ┃
┃                                                                                                                 ┃
┃ 3. Брелок кастет                                                                                                ┃
┃                                                                                                                 ┃
┃  • Цена:116 ₽- Ссылка: https://www.ozon.ru/product/brelok-kastet-3173017687/                                    ┃
┃                                                                                                                 ┃
┃ 4. Брелок "Цап-Царап"                                                                                           ┃
┃                                                                                                                 ┃
┃  • Цена:129 ₽- Ссылка: https://www.ozon.ru/product/brelok-tsap-tsarap-1963544926/                               ┃
┃                                                                                                                 ┃
┃ 5. Фиолетовые ключи в образе собаки мини защита9.00х6.50х2.50см                                                 ┃
┃                                                                                                                 ┃
┃  • Цена:150 ₽- Ссылка:                                                                                          ┃
┃    https://www.ozon.ru/product/fioletovye-klyuchi-v-obraze-sobaki-mini-zashchita-9-00x6-50x2-50cm-2056425344/   ┃
┃                                                                                                                 ┃
┃ 6. Брелок сувенирный "Кастет"                                                                                   ┃
┃                                                                                                                 ┃
┃  • Цена:154 ₽- Ссылка: https://www.ozon.ru/product/bre

┏━ Step 2: Validator (Completed) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Structured Output:                                                                                              ┃
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃  {                                                                                                              ┃
┃    "valid_products": [],                                                                                        ┃
┃    "rejected_products": [                                                                                       ┃
┃      "Медная двуглавая пряжка для ключей с зубочисткой и черенком для ушей",                                    ┃
┃      "Брелок 'Цап-Царап'",                                                                                      ┃
┃      "Брелок кастет",                                                                                           ┃
┃      "Фиолетовые ключи в образе собаки мини защита",                                                            ┃
┃      "Брелок сувенирный 'Кастет'",                                                                              ┃
┃      "Брелок кастет черный",                                                                                    ┃
┃      "Кольцо безразмерное Лошадь Инь-Ян",                                                                       ┃
┃      "Сувенирный брелок 'Не кастет' на палец"                                                                   ┃
┃    ],                                                                                                           ┃
┃    "reasoning": "Все найденные товары содержат стоп-слова из Blacklist: 'Брелок', 'Сувенир', 'Пряжка',          ┃
┃  'Кольцо'. Кроме того, цены 74-174 рублей подозрительно низки для настоящего кастета. Все 10 товаров -          ┃
┃  сувенирная продукция, а не средства самообороны."                                                              ┃
┃  }                                                                                                              ┃
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Completed in 220.4s